In [18]:
import pandas, numpy

## functions & options


In [19]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'

In [20]:
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# ATG7 expression, ensembl. Data preparation 

## 1) Normal tissue + Primary tumors selection, >20 samples

In [21]:
%%time
path = input_file_directory + "xenabrowser_brut_data/xena_surv_ATG7.tsv"
df_ori= pandas.read_csv(path, sep = "\t")
print(df_ori.shape)
df_ori.head()

(19131, 31)
Wall time: 87.8 ms


,sample,samples,_sample_type,_primary_site,ENSG00000197548.12,ENST00000451513.5,ENST00000435760.5,ENST00000451830.5,ENST00000460444.5,ENST00000470474.1,...,ENST00000434066.6,ENST00000478638.5,ENST00000461278.1,ENST00000460291.1,ENST00000467121.1,ENST00000414717.5,ENST00000427759.5,ENST00000446110.1,OS,OS.time
0,TARGET-20-PARUBT-40,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,4.401,-9.9660,0.3685,2.656,-0.7108,-1.0860,...,-9.9660,-9.966,-0.6193,-5.0120,-9.966,0.8568,-9.9660,-9.9660,NaN,NaN
1,TARGET-20-PATJHJ-40,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,2.820,-9.9660,-0.1345,-1.595,-2.9320,-9.9660,...,-9.9660,-9.966,-9.9660,-3.3080,-9.966,-2.0530,-2.7270,-0.2328,NaN,NaN
2,TARGET-10-PASLZM-40,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,1.158,-9.9660,-2.6350,-1.994,-9.9660,-9.9660,...,-9.9660,-9.966,-9.9660,-3.6260,-9.966,-1.0860,-9.9660,-9.9660,NaN,NaN
3,TARGET-21-PATAIJ-42,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White blood cell,4.332,0.2642,1.0850,-1.086,-9.9660,-9.9660,...,-9.9660,-9.966,-9.9660,-1.5950,-9.966,0.3115,-0.6643,1.7230,NaN,NaN
4,TARGET-21-PASVJS-41,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White blood cell,5.156,-9.9660,1.6280,-1.732,-2.8260,-0.6643,...,-0.1187,-9.966,-1.3550,0.4125,-9.966,0.2522,-9.9660,2.4310,NaN,NaN


In [22]:
# Column selection
df = df_ori[['sample',
            '_sample_type',
            '_primary_site',
             'OS', 'OS.time',
            'ENSG00000197548.12',
            'ENST00000354449.7', 
            'ENST00000354956.9',
            'ENST00000446450.6']]

# We have Adrenal Gland and Adrenal gland. I need to put "G".
df = df.assign(_primary_site=df['_primary_site'].str.title())

# Column rename
df = df.rename(columns = {'sample' : 'sample',
                    '_sample_type' : 'Sample_Type',
                    'OS.time' : 'OS_time',
                    '_primary_site' : 'Primary_Site',
                    'ENSG00000197548.12': 'ATG7',
                    'ENST00000354449.7': 'ATG7_1',
                    'ENST00000354956.9': 'ATG7_2',
                    'ENST00000446450.6': 'ATG7_3'})

# Calcul to have value from log2;  2**(ATG7(1))-0.001
df = df.assign(ATG7_total= pow(2,df['ATG7'])-0.001,
          ATG7_1tpm= pow(2,df['ATG7_1'])-0.001,
          ATG7_2tpm= pow(2,df['ATG7_2'])-0.001,
          ATG7_3tpm= pow(2,df['ATG7_3'])-0.001)

# To not have negative values (-0.00000000007) convert it to zero
columns = 'ATG7_total', 'ATG7_1tpm', 'ATG7_2tpm', 'ATG7_3tpm'
for column in columns:
    df[column] = numpy.where(df[column] < 0.0001, 0, df[column])

# Calcul to have log2+1 from value; 
df = df.assign(log2_p1_ATG7_total= numpy.log2(df['ATG7_total']+1),
               log2_p1_ATG7_1= numpy.log2(df['ATG7_1tpm']+1),
               log2_p1_ATG7_2= numpy.log2(df['ATG7_2tpm']+1))

# Calcul to see percentage of expression of the 3 isoforms of ATG7 
df = df.assign(ATG7_prot_tot= df['ATG7_1tpm'] + df['ATG7_2tpm'] + df['ATG7_3tpm'])

df = df.assign(ATG7_1perc= (df['ATG7_1tpm'] *100)/ df['ATG7_prot_tot'],
              ATG7_2perc= (df['ATG7_2tpm'] *100)/ df['ATG7_prot_tot'],
              ATG7_3perc= (df['ATG7_3tpm'] *100)/ df['ATG7_prot_tot'])                              

#Add columns, normalized on total
df = df.assign(ATG7_1norm= df['ATG7_1tpm']/df["ATG7_total"],
               ATG7_2norm= df['ATG7_2tpm']/df["ATG7_total"])                                   

df.head()

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,4.401,1.3900,1.522,-1.7320,21.125765,...,0.300034,4.467655,1.855905,1.952665,5.790710,45.241201,49.577495,5.181305,0.124009,0.135895
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,2.820,1.1180,-1.086,-9.9660,7.060624,...,0.000000,3.010892,1.664236,0.555881,2.639524,82.191275,17.808725,0.000000,0.307262,0.066576
2,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,1.158,0.0158,-3.171,-3.8160,2.230479,...,0.070002,1.691748,1.007204,0.150597,1.190042,84.871949,9.245753,5.882299,0.452823,0.049329
3,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White Blood Cell,NaN,NaN,4.332,1.3450,1.975,-9.9660,20.139115,...,0.000000,4.401843,1.823465,2.301670,6.469584,39.249847,60.750153,0.000000,0.126088,0.195157
4,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White Blood Cell,NaN,NaN,5.156,1.8040,3.769,0.5069,35.653197,...,1.419994,5.195867,2.166995,3.871026,18.542569,18.826249,73.515731,7.658019,0.097912,0.382342


In [23]:
#what kind of samples are the data ? 
df['Sample_Type'].value_counts()

Primary Tumor                                        9185
Normal Tissue                                        7429
Solid Tissue Normal                                   738
Cell Line                                             433
Metastatic                                            393
Primary Solid Tumor                                   286
Primary Blood Derived Cancer - Peripheral Blood       239
Primary Blood Derived Cancer - Bone Marrow            237
Recurrent Blood Derived Cancer - Bone Marrow          104
Recurrent Tumor                                        45
Recurrent Solid Tumor                                  13
Post treatment Blood Cancer - Bone Marrow              12
Additional - New Primary                               11
Recurrent Blood Derived Cancer - Peripheral Blood       3
Control Analyte                                         1
Additional Metastatic                                   1
Post treatment Blood Cancer - Blood                     1
Name: Sample_T

In [24]:
#Creation of two dataframe, Normal tissue and Primary tumor.
df_ATG7_Normal_all = df[df['Sample_Type'].str.match('Normal Tissue')]
df_ATG7_Primary_all = df[df['Sample_Type'].str.match('(Primary Tumor)')]

print('normal_tissue' , len(df_ATG7_Normal_all))
print('primary_tumors' , len(df_ATG7_Primary_all))

normal_tissue 7429
primary_tumors 9185


In [25]:
#Remove tissues with lower than 20 samples. 
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
infos = 'Normal:', 'Primary:'
element_to_delete = []
df_ATG7_filtered_list = []

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    print(info)
    
    #select all the elements in primary sites, and note which one are <20
    working_elements = df["Primary_Site"].to_list()
    for element in working_elements:
        if working_elements.count(element) < 20:
            element_to_delete.append(element)
            
    #Convert to Set() to have a clear list because it cannot contain duplicate. 
    element_to_delete = set(element_to_delete)
    print('elements with less than 20 samples are:', element_to_delete)
    
    #Remove the samples where the tissue is represented less than 20 times
    print('shape with all the elements', df.shape)
    for element in element_to_delete:
        df = df.loc[df["Primary_Site"] != element]
    print('shape after removing the elements', df.shape)
    
    df_ATG7_filtered_list.append(df)
    print()
    
df_ATG7_Normal_all = df_ATG7_filtered_list[0]
df_ATG7_Primary_all = df_ATG7_filtered_list[1]

Normal:
elements with less than 20 samples are: {nan, 'Cervix Uteri', 'Fallopian Tube', 'Bladder'}
shape with all the elements (7429, 22)
shape after removing the elements (7405, 22)

Primary:
elements with less than 20 samples are: {nan, 'Cervix Uteri', 'Fallopian Tube', 'Bladder'}
shape with all the elements (9185, 22)
shape after removing the elements (8778, 22)



In [26]:
#Some samples do not have any primary sites.
df_ATG7_Normal_all.tail()

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
8080,GTEX-WWYW-2826-SM-4MVOQ,Normal Tissue,Uterus,NaN,NaN,3.126,2.566,-9.9660,-9.9660,8.729111,...,0.000000,3.282308,2.790908,0.000000,5.920653,100.000000,0.000000,0.000000,0.678265,0.000000
8081,GTEX-YFC4-0126-101855-SM-5CVLZ,Normal Tissue,NaN,NaN,NaN,4.471,2.888,1.8120,0.2029,22.176118,...,1.150010,4.534567,3.070636,2.173219,12.061732,61.362955,29.102679,9.534366,0.333757,0.158291
8082,GTEX-YEC4-1026-101814-SM-5P9FX,Normal Tissue,NaN,NaN,NaN,3.711,2.275,1.7450,-0.1993,13.094507,...,0.869973,3.817061,2.545715,2.121330,9.059898,53.410937,36.986604,9.602460,0.369542,0.255905
8083,GTEX-YEC3-2526-101809-SM-5CVLT,Normal Tissue,NaN,NaN,NaN,3.609,1.683,1.8720,-0.4325,12.200613,...,0.739977,3.722533,2.073803,2.220143,7.609323,42.184431,48.090963,9.724606,0.263097,0.299935
8084,GTEX-YEC3-1426-101806-SM-5PNXX,Normal Tissue,NaN,NaN,NaN,3.123,2.359,-0.3752,-9.9660,8.710976,...,0.000000,3.279616,2.615686,0.823748,5.899145,86.947286,13.052714,0.000000,0.588814,0.088394


In [27]:
#remove NaN in primary site
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
for df in dfs: 
#     df['Primary_Site'].dropna(axis=0
    df.dropna(subset=['Primary_Site'], inplace=True)
df_ATG7_Normal_all.tail(2)

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
8079,GTEX-ZV6S-1526-SM-5NQ7O,Normal Tissue,Uterus,NaN,NaN,3.173,2.791,-3.047,-9.966,9.018203,...,0.0,3.324552,2.985517,0.16349,7.040087,98.29557,1.70443,0.0,0.767347,0.013306
8080,GTEX-WWYW-2826-SM-4MVOQ,Normal Tissue,Uterus,NaN,NaN,3.126,2.566,-9.966,-9.966,8.729111,...,0.0,3.282308,2.790908,0.00000,5.920653,100.00000,0.00000,0.0,0.678265,0.000000


In [28]:
#all tissues save to csv
path = 'ATG7/TCGA_GTEX/'
path2 = 'all_tissues/dataframes_for_input/'
df_ATG7_Normal_all.to_csv(output_file_directory+path+path2+'df_ATG7_Normal_all.csv',sep = ",", index=False)
df_ATG7_Primary_all.to_csv(output_file_directory+path+path2+'df_ATG7_Primary_all.csv',sep = ",", index=False)

## 2) Filtered data: >20 samples and present in the two groups       (normal tissue & primary tumors)

In [29]:
#How many samples represent each organs?
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
infos = 'normal_tissue', 'primary_tumor'

for df in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    
    print(info.upper())
    print(df["Primary_Site"].value_counts())
    print('___________________________________')

PRIMARY_TUMOR
Breast                     1092
Lung                       1011
Kidney                      884
Brain                       662
Head And Neck Region        518
Thyroid Gland               504
Prostate                    495
Ovary                       419
Stomach                     414
Liver                       369
Cervix                      304
Colon                       288
Soft Tissue,Bone            258
Esophagus                   181
Endometrium                 180
Pancreas                    178
Paraganglia                 177
Testis                      148
Thymus                      119
Skin                        102
Rectum                       92
Lining Of Body Cavities      87
Eye                          79
Adrenal Gland                77
Uterus                       57
Lymphatic Tissue             47
Bile Duct                    36
Name: Primary_Site, dtype: int64
___________________________________
PRIMARY_TUMOR
Breast                     1092
Lung   

In [30]:
#Change names to regroup Thyroid and Thyroid Gland
df_ATG7_Primary_all['Primary_Site'].replace('Thyroid Gland', 'Thyroid', inplace=True)
#to see what tissues are present in both group
norm_sites = df_ATG7_Normal_all['Primary_Site']
prim_sites = df_ATG7_Primary_all['Primary_Site']

set1 = set(norm_sites)
set2 = set(prim_sites)
inter_site = set1.intersection(set2)
print('tissues in the two groups:')
inter_site

tissues in the two groups:


{'Adrenal Gland',
 'Brain',
 'Breast',
 'Colon',
 'Esophagus',
 'Kidney',
 'Liver',
 'Lung',
 'Ovary',
 'Pancreas',
 'Prostate',
 'Skin',
 'Stomach',
 'Testis',
 'Thyroid',
 'Uterus'}

In [31]:
#Select only tissues present in the two groups
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
dfs_inter_L = []
for df in dfs:
    df_inter = df[df['Primary_Site'].isin(inter_site)]
    dfs_inter_L.append(df_inter)
    
df_ATG7_Normal_filt = dfs_inter_L[0]
df_ATG7_Primary_filt = dfs_inter_L[1]
print(df_ATG7_Primary_filt["Primary_Site"].value_counts())

#save to csv
path2 = 'all_filtered_tissues/dataframes_for_input/'
df_ATG7_Normal_filt.to_csv(output_file_directory+path+path2+'df_ATG7_Normal_filtered.csv',sep = ",", index=False)
df_ATG7_Primary_filt.to_csv(output_file_directory+path+path2+'df_ATG7_Primary_filtered.csv',sep = ",", index=False)

Breast           1092
Lung             1011
Kidney            884
Brain             662
Thyroid           504
Prostate          495
Ovary             419
Stomach           414
Liver             369
Colon             288
Esophagus         181
Pancreas          178
Testis            148
Skin              102
Adrenal Gland      77
Uterus             57
Name: Primary_Site, dtype: int64


In [32]:
df_ATG7_Normal_filt.head(5)

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
1707,GTEX-132AR-1726-SM-5EGHQ,Normal Tissue,Adrenal Gland,NaN,NaN,4.496,2.803,1.614,-0.2498,22.563767,...,0.840013,4.558498,2.996009,2.021477,10.877908,64.147459,28.130349,7.722192,0.309253,0.135615
1708,GTEX-U4B1-0126-SM-4DXSN,Normal Tissue,Adrenal Gland,NaN,NaN,4.293,2.958,1.860,-0.0725,19.601965,...,0.949989,4.364710,3.132488,2.210724,12.348525,62.918120,29.388745,7.693134,0.396361,0.185138
1709,GTEX-11TT1-0126-SM-5LUAA,Normal Tissue,Adrenal Gland,NaN,NaN,4.271,2.862,2.161,0.2029,19.305303,...,1.150010,4.343785,3.047752,2.451870,12.890482,56.392188,34.686425,8.921386,0.376540,0.231607
1710,GTEX-O5YT-1326-SM-3MJGR,Normal Tissue,Adrenal Gland,NaN,NaN,4.224,3.098,1.496,-0.2159,18.686478,...,0.860009,4.299133,3.257208,1.933420,12.240914,69.940115,23.034194,7.025691,0.458155,0.150890
1711,GTEX-11NSD-0226-SM-5A5LR,Normal Tissue,Adrenal Gland,NaN,NaN,4.218,2.945,1.665,-1.5950,18.608921,...,0.330022,4.293438,3.120975,2.060095,11.199914,68.748343,28.305007,2.946650,0.413767,0.170356


## 3) Creation of Normal-Primary dataframe

### a) all tissues >20 samples

In [33]:
# create df with only Normal and Primary 
df_ATG7_NormPrim_all = pandas.concat([df_ATG7_Normal_all, df_ATG7_Primary_all], axis=0)
df_ATG7_NormPrim_all.reset_index(drop=True, inplace=True)

# #remove Survival in formation
# df_ATG7_NormPrim_all.drop(df_ATG7_NormPrim_all.columns[[3, 4]], axis = 1, inplace = True)
# print(df_ATG7_NormPrim_all.shape)

#save to csv
path2 = 'all_tissues/dataframes_for_input/'
df_ATG7_NormPrim_all.to_csv(output_file_directory+path+path2+'df_ATG7_NormPrim_all.csv',sep = ",", index=False)


### b) filtered tissues (>20 samples and tissues present in the two groups)

In [34]:
# create df with only Normal and Primary 
df_ATG7_NormPrim_filt = pandas.concat([df_ATG7_Normal_filt, df_ATG7_Primary_filt], axis=0)
df_ATG7_NormPrim_filt.reset_index(drop=True, inplace=True)

# #remove Survival in formation
# df_ATG7_NormPrim_filt.drop(df_ATG7_NormPrim_filt.columns[[3, 4]], axis = 1, inplace = True)
# print(df_ATG7_NormPrim_filt.shape)

#save to csv
path2 = 'all_filtered_tissues/dataframes_for_input/'
df_ATG7_NormPrim_filt.to_csv(output_file_directory+path+path2+'df_ATG7_NormPrim_filtered.csv',sep = ",", index=False)
